# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
df=cities.iloc[:-1,[0,3,5,6,7,8]]
df =df.rename(columns={'Population (2016 est.)[8]': 'Population'})

def matching_values(sport, region):
    sport['team'] = sport['team'].replace('([^ \w]*)','', regex=True)

    sport['reg'] = sport['team']
    for reg in region['Metropolitan area']:                               # this loop will match the team name with the area name
                                                                        # return the area or region name, i can create a function on this
            for team in sport['reg']:
                if not reg.startswith(team[:5]) : continue
                sport['reg'] = sport['reg'].replace(to_replace=team, value=reg)
    return sport

def nhl_correlation(): 
    # YOUR CODE HERE
    global df_nhl, nhl, df
    nhl = nhl[nhl['year'] == 2018]
    
    nhl['team'] = nhl['team'].replace('([^ \w].*)','', regex=True)     #replace any value other than letter like *
    
    nhl['reg'] = nhl['team']                                          # in order to work on team column
    
    for reg in df['Metropolitan area']:                               # this loop will match the team name with the area name
                                                                    # return the area or region name, i can create a function on this
        for team in nhl['reg']:
            if not reg.startswith(team[:5]) : continue
            nhl['reg'] = nhl['reg'].replace(to_replace=team, value=reg)
            
    nhl['reg'] = nhl['reg'].replace({"St Louis Blues" : "St. Louis",                # some hardcoded name where name doesnt't match with area name
                   "Florida Panthers": "Miami–Fort Lauderdale",
                   'Carolina Hurricanes' : 'Raleigh',
                   "San Jose Sharks" : 'San Francisco Bay Area',
                   'New Jersey Devils': "New York City",
                   'Colorado Avalanche': "Denver",
                   "Vegas Golden Knights" : "Las Vegas",
                   "Arizona Coyotes": "Phoenix" ,
                    "Anaheim Ducks": "Los Angeles"})
    
    df_nhl = pd.merge(df, nhl, left_on="Metropolitan area", right_on='reg', how='inner')       # merge the two datasets
    
    df_nhl = df_nhl.loc[:,['Metropolitan area','Population', 'team', 'W', "L"]]                # get the required data columns
    
    df_nhl = df_nhl[df_nhl['team'] != 'Atlantic Division']                                    # all values other than atlantic value which is all over
                                                                                                # the data frame
    
    df_nhl[['Population', 'W',"L" ]] = df_nhl.loc[:, ['Population', "W", "L"]].apply(pd.to_numeric, errors='coerce')
    # convert the string to int for calculation
    
    # group by as requirement
    df_nhl = df_nhl.groupby(["Metropolitan area", 'Population']).agg({'W': np.sum,
                                                 "L": np.sum},axis =0)
    
    # reset index where i can slice data easily
    df_nhl.reset_index(inplace=True)
    df_nhl['total'] = df_nhl.loc[:, ['W','L']].apply(lambda x: np.sum(x), axis =1)
    df_nhl['win_loss_ratio'] = df_nhl['W']/ df_nhl['total']
    
    #return df_nhl['Population'].sort_values().corr(df_nhl['win_loss_ratio'])

    #raise NotImplementedError()
    
    population_by_region = df_nhl["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = df_nhl["win_loss_ratio"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()
#stats.pearsonr(df_nhl['Population'], df_nhl['win_loss_ratio'])

0.012308996455744249

In [4]:
import random
def estimate_pie(n):
    num_point_circle = 0
    total_point_circle = 0
    for _ in range(n):
        x = random.uniform(0,1)
        y = random.uniform(0,1)
        distance = x**2 + y**2
        if distance <= 1:
            num_point_circle+=1
        total_point_circle+=1
        #print(num_point_circle)
    return 4 * num_point_circle/total_point_circle
estimate_pie(1000000)
            


3.13868

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba=pd.read_csv("assets/nba.csv")
df=pd.read_html("assets/wikipedia_data.html")[1]
df=cities.iloc[:-1,[0,3,5,6,7,8]]
df =df.rename(columns={'Population (2016 est.)[8]': 'Population'})
def nba_correlation():
    # YOUR CODE HERE
    global nba_df, nba, df
    
    df['Population'] = df['Population'].apply(pd.to_numeric, errors='coerce')
    
    nba = nba[nba['year'] == 2018]
    nba.loc[:,['W','L',"W/L%"]] = nba.loc[:,['W','L',"W/L%"]].apply(pd.to_numeric, errors='coerce')
    nba['team']  = nba['team'].replace({"([^\w ]*\(\d\))" :  "",
                                   "([ ]\(\d*)" : ""}, regex=True)
    
    nba = matching_values(nba, df)
    
    nba['reg'] = nba['reg'].replace({"Golden State Warriors": "San Francisco Bay Area",
                                "(Utah Jazz\d*)": "Salt Lake City",
                                "Brooklyn Nets\d*": "New York City"}, regex= True)
    
    nba_df = pd.merge(df, nba, left_on= "Metropolitan area", right_on='reg').loc[:,['Metropolitan area', "Population", 'W', "L"]]
    nba_df = nba_df.groupby(['Metropolitan area', 'Population']).agg({"W": np.sum,
                                                                     "L": np.sum}, axis = 1)
    nba_df.reset_index(inplace=True)
    nba_df['total'] = nba_df['W'] + nba_df['L']
    nba_df['win_loss_ratio'] = nba_df['W'] / nba_df['total']
    
    
    
    #raise NotImplementedError()
    
    population_by_region = nba_df["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = nba_df["win_loss_ratio"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
df=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    global df, mlb, mlb_df
    # clean the Population columns name:
    df = pd.read_html('assets/wikipedia_data.html')[1]
    df = df.rename(columns={"Population (2016 est.)[8]": 'Population'})

    # clean the population columns


    mlb = pd.read_csv('assets/mlb.csv')
    #
    # only for 2018
    mlb = mlb[mlb['year'] == 2018]

    # cleaning and matching the row value and call other value
    mlb = matching_values(mlb, df)

    # clean the value with hard coding
    mlb['reg'] = mlb['reg'].replace({"Oakland Athletics" : "San Francisco Bay Area",
                                    "Texas Rangers"  : "Dallas–Fort Worth",
                                    "Colorado Rockies": "Denver",
                                    "Arizona Diamondbacks" : "Phoenix",
                                     "St Louis Cardinals" : "St. Louis"
                                    })

    # merge the dataframes and take the required columns
    mlb_df = pd.merge(df, mlb , right_on ="reg", left_on='Metropolitan area', how='inner').loc[:, ["Metropolitan area"
                                                                                                    ,"Population",
                                                                                                    "W",
                                                                                                   "L"]]
    # change the dtype to all numeric in these columns
    mlb_df[["Population","W","L"]]  = mlb_df.loc[:,["Population","W","L"]].apply(pd.to_numeric, errors='coerce')

    # group by
    mlb_df = mlb_df.groupby(['Metropolitan area', "Population"]).agg({"W": np.sum,
                                                            "L" : np.sum}, axis = 0)

    mlb_df.reset_index(inplace=True)
    mlb_df['win_loss_ratio']  = mlb_df['W'] / (mlb_df['W'] + mlb_df['L'])
    
    #raise NotImplementedError()
    
    population_by_region = mlb_df["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = mlb_df["win_loss_ratio"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()


0.1505230448710485

In [23]:
mlb
[x for x in mlb['reg'] if "St" in x]

['St Louis Cardinals']

In [7]:
df

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
df=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    #lets clean the region NFL columns
    global df, nfl, nfl_df
    
    df = df.rename(columns={"Population (2016 est.)[8]": "Population"})

    df['NFL'] = df['NFL'].replace({'(\[note[ \w]*])': "",
                                 "[^\w]*": ""}, regex=True)
    df = df[df['NFL'] !=""]                                         # its redundency
    
    nfl['team'] = nfl['team'].replace("([^ \w]*)", "", regex=True)
    nfl = nfl[nfl['year'] == 2018]
    nfl = matching_values(nfl, df)
    
    # replce the additonal team name in reg into region_name who doesn't match with region
    nfl['reg'] = nfl['reg'].replace({"Arizona Cardinals": "Phoenix",
                                "Carolina Panthers": "Charlotte",
                                "New England Patriots": "Boston",
                                 "Oakland Raiders":  "San Francisco Bay Area",
                                "Tennessee Titans" : "Nashville"})
    
    # merge and change the dtype of required columns
    nfl_df = df.merge(nfl, left_on="Metropolitan area", right_on = "reg", how="inner")

    nfl_df = nfl_df[['Metropolitan area', "Population", "W", "L", "W-L%" ]]
    nfl_df[["Population", "W", "L", "W-L%"]] = nfl_df[["Population", "W", "L", "W-L%"]].apply(pd.to_numeric, errors='coerce')
    
    # groupby for get per region wise population, win, loss
    nfl_df = nfl_df.groupby(['Metropolitan area', "Population"]).agg({"W": np.sum,
                                                                 "L": np.sum,
                                                                 "W-L%": np.nanmean}, axis=0)
    nfl_df.reset_index(inplace=True)
    
    
    #raise NotImplementedError()
    
    population_by_region = nfl_df["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = nfl_df["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

0.004282141436392957

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# mlb_df=pd.read_csv("assets/mlb.csv")
# nhl_df=pd.read_csv("assets/nhl.csv")
# nba_df=pd.read_csv("assets/nba.csv")
# nfl_df=pd.read_csv("assets/nfl.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    global nfl_nba, nfl_nhl, df_nhl, nhl_df, mlb_df, nfl_df, nba_df
    # YOUR CODE HERE
    nhl_df = df_nhl
    
    nfl_df['win_loss_ratio'] = nfl_df['W-L%']
    
    def join_4dataset(df1, df2, df3, df4):
        df1_df1 = df1.merge(df1, on='Metropolitan area', how='inner')
        df1_df2 = df1.merge(df2, on='Metropolitan area', how='inner')
        df1_df3 = df1.merge(df3, on='Metropolitan area', how='inner')
        df1_df4 = df1.merge(df4, on='Metropolitan area', how='inner')

        return df1_df1, df1_df2, df1_df3, df1_df4
    
    def p_value(df1, df2, df3, df4):
        p_value = []
        lst = [df1, df2, df3, df4]
        for df in lst:
            p_value.append(stats.ttest_rel(df['win_loss_ratio_x'], df['win_loss_ratio_y'])[1])
        return p_value
    
    
    nhl_nhl, nhl_nba, nhl_mlb, nhl_nfl   =join_4dataset(nhl_df, nba_df, mlb_df, nfl_df)
    nba_nba, nba_nhl, nba_mlb, nba_nfl   = join_4dataset(nba_df, nhl_df,  mlb_df, nfl_df)
    mlb_mlb, mlb_nhl, mlb_nba, mlb_nfl =  join_4dataset(mlb_df, nhl_df, nba_df , nfl_df)
    nfl_nfl, nfl_nhl, nfl_nba, nfl_mlb  = join_4dataset(nfl_df, nhl_df, nba_df, mlb_df)
    
    p_values = pd.DataFrame({"NFL": p_value(nfl_nfl, nfl_nba, nfl_nhl, nfl_mlb),
                             "NBA" : p_value(nba_nfl, nba_nba, nba_nhl, nba_mlb),
                            "NHL": p_value(nhl_nfl, nhl_nba, nhl_nhl, nhl_mlb),
                             "MLB" : p_value(mlb_nfl, mlb_nba, mlb_nhl, mlb_mlb)
                                 }, index = ['NFL', 'NBA', 'NHL', 'MLB'])
    
#     p_values = pd.DataFrame({"NHL": p_value(nhl_nhl, nhl_nba, nhl_mlb, nhl_nfl),
#                   "NBA" : p_value(nba_nhl, nba_nba, nba_mlb, nba_nfl),
#                   "MLB" : p_value(mlb_nhl, mlb_nba, mlb_mlb, mlb_nfl),
#                   "NFL": p_value(nfl_nhl, nfl_nba, nfl_mlb, nfl_nfl)}, index = ['NHL',"NBA", 'MLB', "NFL"])
    
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    #sports = ['NFL', 'NBA', 'NHL', 'MLB']
    #p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values.astype(dtype='float64')
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937445,0.030392,0.803345
NBA,0.937445,NaN,0.022316,0.951046
NHL,0.030392,0.022316,NaN,0.000712
MLB,0.803345,0.951046,0.000712,NaN


In [15]:
nhl_df = df_nhl
    
nfl_df['win_loss_ratio'] = nfl_df['W-L%']

def join_4dataset(df1, df2, df3, df4):
    df1_df1 = df1.merge(df1, on='Metropolitan area', how='inner')
    df1_df2 = df1.merge(df2, on='Metropolitan area', how='inner')
    df1_df3 = df1.merge(df3, on='Metropolitan area', how='inner')
    df1_df4 = df1.merge(df4, on='Metropolitan area', how='inner')

    return df1_df1, df1_df2, df1_df3, df1_df4

def p_value(df1, df2, df3, df4):
        p_value = []
        lst = [df1, df2, df3, df4]
        for df in lst:
            p_value.append(stats.ttest_rel(df['win_loss_ratio_x'], df['win_loss_ratio_y'])[1])
        return p_value
    
nhl_nhl, nhl_nba, nhl_mlb, nhl_nfl   =join_4dataset(nhl_df, nba_df, mlb_df, nfl_df)
nba_nba, nba_nhl, nba_mlb, nba_nfl   = join_4dataset(nba_df, nhl_df,  mlb_df, nfl_df)
mlb_mlb, mlb_nhl, mlb_nba, mlb_nfl =  join_4dataset(mlb_df, nhl_df, nba_df , nfl_df)
nfl_nfl, nfl_nhl, nfl_nba, nfl_mlb  = join_4dataset(nfl_df, nhl_df, nba_df, mlb_df)

p_values = pd.DataFrame({"NFL": p_value(nfl_nfl, nfl_nba, nfl_nhl, nfl_mlb),
                             "NBA" : p_value(nba_nfl, nba_nba, nba_nhl, nba_mlb),
                            "NHL": p_value(nhl_nfl, nhl_nba, nhl_nhl, nhl_mlb),
                             "MLB" : p_value(mlb_nfl, mlb_nba, mlb_nhl, mlb_mlb)
                                 }, index = ['NFL', 'NBA', 'NHL', 'MLB'])
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.937445,0.030392,0.803345
NBA,0.937445,NaN,0.022316,0.951046
NHL,0.030392,0.022316,NaN,0.000712
MLB,0.803345,0.951046,0.000712,NaN


In [27]:
# sports = ['NFL', 'NBA', 'NHL', 'MLB']
# p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
# p_values
nba_df

,Metropolitan area,Population,W,L,total,win_loss_ratio
0,Atlanta,5789700,24,58,82,0.292683
1,Boston,4794447,55,27,82,0.670732
2,Charlotte,2474314,36,46,82,0.439024
3,Chicago,9512999,27,55,82,0.329268
4,Cleveland,2055612,50,32,82,0.609756
5,Dallas–Fort Worth,7233323,24,58,82,0.292683
6,Denver,2853077,46,36,82,0.560976
7,Detroit,4297617,39,43,82,0.475610
8,Houston,6772470,65,17,82,0.792683
9,Indianapolis,2004230,48,34,82,0.585366


In [14]:
print(len(nfl_nhl))
print(len(nfl_nba))

17
19


In [18]:
stats.ttest_rel(nfl_nba['win_loss_ratio_x'], nfl_nba['win_loss_ratio_y'])

Ttest_relResult(statistic=0.07958546013371386, pvalue=0.9374450970928311)

In [29]:
nfl_nhl

,Metropolitan area,Population_x,W_x,L_x,W-L%,win_loss_ratio_x,Population_y,W_y,L_y,total,win_loss_ratio_y
0,Boston,4794447,11,5,0.6880,0.6880,4794447,50,20,70,0.714286
1,Buffalo,1132804,6,10,0.3750,0.3750,1132804,25,45,70,0.357143
2,Chicago,9512999,12,4,0.7500,0.7500,9512999,33,39,72,0.458333
3,Dallas–Fort Worth,7233323,10,6,0.6250,0.6250,7233323,42,32,74,0.567568
4,Denver,2853077,6,10,0.3750,0.3750,2853077,43,30,73,0.589041
5,Detroit,4297617,6,10,0.3750,0.3750,4297617,30,39,69,0.434783
6,Los Angeles,13310447,25,7,0.7815,0.7815,13310447,89,54,143,0.622378
7,Miami–Fort Lauderdale,6066387,7,9,0.4380,0.4380,6066387,44,30,74,0.594595
8,Minneapolis–Saint Paul,3551036,8,7,0.5310,0.5310,3551036,45,26,71,0.633803
9,Nashville,1865298,9,7,0.5630,0.5630,1865298,53,18,71,0.746479
